In [129]:
# install pakages
!pip install --upgrade -q langchain-google-genai

In [130]:
from google.colab import userdata
# GEMINI API KEY
GOOGLE_API_KEY= userdata.get("GOOGLE_API_KEY")
# WETAHTER API KEY
WEATHER_API_KEY= userdata.get("WEATHER_API_KEY")

In [131]:
from langchain_google_genai import ChatGoogleGenerativeAI

In [132]:
# configure model and
llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash-exp",
    api_key=GOOGLE_API_KEY # GEMINI API KEY
)

In [133]:
from langchain_core.tools import tool
import requests

# decorator
@tool
def annual_return(a:int):
  """Returns annual return of a number"""
  return (a +15)/5

# decorator
@tool
def add_two_numbers(a:int, b :int):
  """Returns additional return of a number""" # this doc string
  return (a +b)


# decorator
@tool
def multi(a:int, b:int):
    """Returns multiply return of a number"""
    return( a*b)






@tool
def get_weather_data(city: str):
    """
    Fetches weather data for the given city using OpenWeatherMap API.

    Args:
        city (str): The name of the city to get the weather for.
        api_key (str): Your OpenWeatherMap API key.

    Returns:
        dict: A dictionary containing weather data for the city.
    """
    base_url = f"https://api.openweathermap.org/data/2.5/weather"
    params = {
        "q": city,
        "appid": WEATHER_API_KEY,
        "units": "metric"  # Use "imperial" for Fahrenheit
    }

    try:
        response = requests.get(base_url, params=params)
        response.raise_for_status()  # Raise an HTTPError for bad responses
        weather_data = response.json()
        return weather_data
    except requests.exceptions.RequestException as e:
        return {"error": str(e)}



In [134]:
tools = [ annual_return, multi , add_two_numbers,get_weather_data]

In [101]:
llm_with_tools = llm.bind_tools(tools)

In [102]:
response = llm_with_tools.invoke("What is the anual return  of 109?")

response

AIMessage(content='', additional_kwargs={'function_call': {'name': 'annual_return', 'arguments': '{"a": 109.0}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': [{'category': 'HARM_CATEGORY_HATE_SPEECH', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_DANGEROUS_CONTENT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HARASSMENT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT', 'probability': 'NEGLIGIBLE', 'blocked': False}]}, id='run-3c320352-67cc-4971-a52d-4b0f1e590ef4-0', tool_calls=[{'name': 'annual_return', 'args': {'a': 109.0}, 'id': '625b7ac7-1fc5-40cb-a825-b27a18da89b0', 'type': 'tool_call'}], usage_metadata={'input_tokens': 248, 'output_tokens': 4, 'total_tokens': 252, 'input_token_details': {'cache_read': 0}})

In [153]:
from langchain_core.messages import HumanMessage, AIMessage,ToolMessage
query = "Q1 What is the weather of lahore ?"

message = [HumanMessage(query)]

display(message)

[HumanMessage(content='Q1 What is the weather of lahore ?', additional_kwargs={}, response_metadata={})]

In [154]:
ai_msg = llm_with_tools.invoke(message)
message.append(tools)
message

[HumanMessage(content='Q1 What is the weather of lahore ?', additional_kwargs={}, response_metadata={}),
 [StructuredTool(name='annual_return', description='Returns annual return of a number', args_schema=<class 'langchain_core.utils.pydantic.annual_return'>, func=<function annual_return at 0x7f1063113b50>),
  StructuredTool(name='multi', description='Returns multiply return of a number', args_schema=<class 'langchain_core.utils.pydantic.multi'>, func=<function multi at 0x7f1063110040>),
  StructuredTool(name='add_two_numbers', description='Returns additional return of a number', args_schema=<class 'langchain_core.utils.pydantic.add_two_numbers'>, func=<function add_two_numbers at 0x7f1063110c10>),
  StructuredTool(name='get_weather_data', description='Fetches weather data for the given city using OpenWeatherMap API.\n\nArgs:\n    city (str): The name of the city to get the weather for.\n    api_key (str): Your OpenWeatherMap API key.\n\nReturns:\n    dict: A dictionary containing weat

In [156]:
print(ai_msg.tool_calls)

[{'name': 'get_weather_data', 'args': {'city': 'lahore'}, 'id': '4df81f77-ad40-4642-ab5f-63d47552498f', 'type': 'tool_call'}]


In [157]:
ai_msg.tool_calls

[{'name': 'get_weather_data',
  'args': {'city': 'lahore'},
  'id': '4df81f77-ad40-4642-ab5f-63d47552498f',
  'type': 'tool_call'}]

whar is the drawbaks of ai tool calling

AST_summery means  (abstract syntex tree)

In [158]:
# Invoke the function / tool
for tool_call in ai_msg.tool_calls:
    selected_tool = {
        "add_two_numbers": add_two_numbers,
        "multi": multi,
        "get_weather_data": get_weather_data,
        "annual_return":annual_return
      }[tool_call["name"].lower()]
    tool_msg = selected_tool.invoke(tool_call)
    display(tool_msg)
    message.append(tool_msg)

ToolMessage(content='{"coord": {"lon": 74.3436, "lat": 31.5497}, "weather": [{"id": 711, "main": "Smoke", "description": "smoke", "icon": "50n"}], "base": "stations", "main": {"temp": 11.99, "feels_like": 11.23, "temp_min": 11.99, "temp_max": 11.99, "pressure": 1019, "humidity": 76, "sea_level": 1019, "grnd_level": 993}, "visibility": 4000, "wind": {"speed": 0, "deg": 0}, "clouds": {"all": 0}, "dt": 1736793623, "sys": {"type": 1, "id": 7585, "country": "PK", "sunrise": 1736733760, "sunset": 1736770765}, "timezone": 18000, "id": 1172451, "name": "Lahore", "cod": 200}', name='get_weather_data', tool_call_id='4df81f77-ad40-4642-ab5f-63d47552498f')

what is fine tuning
crafting means from scratch
refining means bar bar wahi process

explecit or implecit

question or instruction this is zero short

1 example 1 short

examples then few short


chain of thouths
Zero-Shot Chain-of-Thought








In [166]:

display(tool_msg.content)

'{"coord": {"lon": 74.3436, "lat": 31.5497}, "weather": [{"id": 711, "main": "Smoke", "description": "smoke", "icon": "50n"}], "base": "stations", "main": {"temp": 11.99, "feels_like": 11.23, "temp_min": 11.99, "temp_max": 11.99, "pressure": 1019, "humidity": 76, "sea_level": 1019, "grnd_level": 993}, "visibility": 4000, "wind": {"speed": 0, "deg": 0}, "clouds": {"all": 0}, "dt": 1736793623, "sys": {"type": 1, "id": 7585, "country": "PK", "sunrise": 1736733760, "sunset": 1736770765}, "timezone": 18000, "id": 1172451, "name": "Lahore", "cod": 200}'

In [168]:

# Invoke the llm
response = llm_with_tools.invoke(tool_msg.content)



AIMessage(content='The weather in Lahore is smoke with a temperature of 11.99 degrees Celsius. The wind speed is 0 and the humidity is 76%.', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': [{'category': 'HARM_CATEGORY_HATE_SPEECH', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_DANGEROUS_CONTENT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HARASSMENT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT', 'probability': 'NEGLIGIBLE', 'blocked': False}]}, id='run-4f66fa1e-7f96-4d65-b75d-fc219e3437c3-0', usage_metadata={'input_tokens': 528, 'output_tokens': 34, 'total_tokens': 562, 'input_token_details': {'cache_read': 0}})

In [169]:
from IPython.display import Markdown

Markdown(response.content)

The weather in Lahore is smoke with a temperature of 11.99 degrees Celsius. The wind speed is 0 and the humidity is 76%.